# Setup and load in the data

In [1]:
# Data handling
import pandas as pd
import numpy as np

# Machine learning model and splitting the data
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Metrics for evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data (replace 'file.csv' with your actual file path)
complete_df = pd.read_csv('Resources/cleaned_completed_data.csv')

# Convert 'Date' column to datetime format
complete_df['date'] = pd.to_datetime(complete_df['date'], errors='coerce')

# Display the results
complete_df.head()

,date,index_sa,redfin_hpi_mom,case_shiller_index_mom,period_duration,region_type,table_id,is_seasonally_adjusted,region,state,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,30_year_%,price_drops_is_blank,price_drops_mom_is_blank,price_drops_yoy_is_blank
0,2012-01-01,59.9,0.0,-0.000005,30,state,23,False,Oklahoma,Oklahoma,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.915,True,True,True
1,2012-01-01,59.9,0.0,-0.000005,30,state,10,False,New Hampshire,New Hampshire,...,0.000000,0.000000,0.0,0.007093,0.000006,-0.000002,3.915,True,True,True
2,2012-01-01,59.9,0.0,-0.000005,30,state,42,False,Virginia,Virginia,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.915,True,True,True
3,2012-01-01,59.9,0.0,-0.000005,30,state,47,False,Michigan,Michigan,...,0.000000,0.000000,0.0,0.013045,-0.000005,0.000007,3.915,True,True,True
4,2012-01-01,59.9,0.0,-0.000005,30,state,12,False,New Jersey,New Jersey,...,0.000549,0.000005,0.0,0.007897,0.000018,0.000061,3.915,False,False,True


In [3]:
# Convert categorical columns using one-hot encoding
complete_df = pd.get_dummies(complete_df, columns=['region_type', 'is_seasonally_adjusted', 
                                                   'region', 'state', 'property_type'], drop_first=True)

In [4]:
# Select numeric columns from the DataFrame
numeric_df = complete_df.select_dtypes(include=[np.number])

# Show the min and max values for all columns
for col in numeric_df.columns:
    print(f"{col}: Min = {numeric_df[col].min()}, Max = {numeric_df[col].max()}")

index_sa: Min = 59.9, Max = 172.1
redfin_hpi_mom: Min = -0.0012, Max = 0.019
case_shiller_index_mom: Min = -6.97e-05, Max = 0.000189
period_duration: Min = 30, Max = 30
table_id: Min = 1, Max = 51
median_sale_price: Min = 7000, Max = 3275000
median_sale_price_mom: Min = -0.00913, Max = 0.2215270935960591
median_sale_price_yoy: Min = -0.0094134677419354, Max = 0.1257142857142857
median_list_price: Min = 53000.0, Max = 4700000.0
median_list_price_mom: Min = -0.0074388387547208, Max = 0.0357864417134923
median_list_price_yoy: Min = -0.0072507525002704, Max = 0.0363052617609819
median_ppsf: Min = 3.0, Max = 40906.0
median_ppsf_mom: Min = -0.0099740143148445, Max = 3.12172894350185
median_ppsf_yoy: Min = -0.0099760368452768, Max = 2.4325566423457765
median_list_ppsf: Min = 27.0, Max = 1873.0
median_list_ppsf_mom: Min = -0.0095540224646076, Max = 0.201536630306659
median_list_ppsf_yoy: Min = -0.0094918034612375, Max = 0.2102027881762564
homes_sold: Min = 1, Max = 53605
homes_sold_mom: Min = 

In [5]:
# Fill any missing values with the mean
complete_df = complete_df.fillna(complete_df.mean())

# Confirm completed
complete_df.isnull().sum()

date                                       0
index_sa                                   0
redfin_hpi_mom                             0
case_shiller_index_mom                     0
period_duration                            0
                                          ..
state_Wyoming                              0
property_type_Condo/Co-op                  0
property_type_Multi-Family (2-4 Unit)      0
property_type_Single Family Residential    0
property_type_Townhouse                    0
Length: 156, dtype: int64

## Preprocessing

In [6]:
# Inspect 'median_ppsf' due to large range

# Define bins in increments of $0–75
bins = np.arange(0, numeric_df['median_ppsf'].max() + 75, 75)
labels = [f"${int(b)}-${int(b + 75)}" for b in bins[:-1]]

# Bin the 'median_ppsf' column
numeric_df['median_ppsf_bin'] = pd.cut(numeric_df['median_ppsf'], bins=bins, labels=labels, include_lowest=True)

# Count the occurrences in each bin
bin_counts = numeric_df['median_ppsf_bin'].value_counts().sort_index()

# Display the counts
bin_counts.head(25)

median_ppsf_bin
$0-$75          3668
$75-$150       18662
$150-$225       9250
$225-$300       3680
$300-$375       1449
$375-$450        974
$450-$525        635
$525-$600        357
$600-$675        133
$675-$750        112
$750-$825         44
$825-$900         16
$900-$975         10
$975-$1050        14
$1050-$1125        7
$1125-$1200       11
$1200-$1275        2
$1275-$1350        4
$1350-$1425        7
$1425-$1500        3
$1500-$1575        0
$1575-$1650        2
$1650-$1725        1
$1725-$1800        0
$1800-$1875        1
Name: count, dtype: int64

In [7]:
# Taking the bin_counts find the highest 25 'median_ppsf' values

# Sort the median_ppsf column in descending order and find the top 5 highest values
top_25_values = numeric_df['median_ppsf'].sort_values(ascending=False).head(25)

print("Highest 25 median_ppsf values:")
top_25_values

Highest 25 median_ppsf values:


38340    40906.0
8121     25964.0
6345     12690.0
16296     7214.0
21969     3790.0
3339      3672.0
1774      3140.0
19507     2712.0
31140     2504.0
14506     1996.0
16246     1957.0
9459      1918.0
3260      1839.0
22422     1720.0
8381      1645.0
16531     1576.0
2679      1468.0
17529     1463.0
2825      1457.0
6147      1409.0
29086     1406.0
15701     1395.0
3311      1393.0
9820      1375.0
22875     1371.0
Name: median_ppsf, dtype: float64

In [8]:
# Taking the bin_counts find the lowest 25 'median_ppsf' values

# Sort the median_ppsf column in ascending order and find the top 25 lowest values
bottom_25_values = numeric_df['median_ppsf'].sort_values(ascending=True).head(25)

print("Lowest 25 median_ppsf values:")
bottom_25_values

Lowest 25 median_ppsf values:


29546     3.0
11118     5.0
32715     6.0
35896     7.0
53        7.0
16735     8.0
20001     8.0
13895     8.0
1577      8.0
21080     9.0
1137      9.0
6841      9.0
26656    10.0
111      10.0
36455    10.0
29422    10.0
1394     11.0
13379    11.0
34766    12.0
14304    12.0
16598    12.0
16994    13.0
426      13.0
61       13.0
20297    13.0
Name: median_ppsf, dtype: float64

In [9]:
# Define bins for $100k increments
bins = np.arange(0, numeric_df['median_list_price'].max() + 100000, 100000)
labels = [f"${int(b)}-${int(b + 100000)}" for b in bins[:-1]]

# Bin the 'median_list_price' column
numeric_df['median_list_price_bin'] = pd.cut(numeric_df['median_list_price'], bins=bins, labels=labels, include_lowest=True)

# Count the occurrences in each bin
list_price_bin_counts = numeric_df['median_list_price_bin'].value_counts().sort_index()

# Display the counts
list_price_bin_counts

median_list_price_bin
$0-$100000              77
$100000-$200000       9697
$200000-$300000      14605
$300000-$400000       6954
$400000-$500000       3679
$500000-$600000       2118
$600000-$700000       1014
$700000-$800000        477
$800000-$900000        216
$900000-$1000000       104
$1000000-$1100000       27
$1100000-$1200000       16
$1200000-$1300000        7
$1300000-$1400000        5
$1400000-$1500000        1
$1500000-$1600000       10
$1600000-$1700000        4
$1700000-$1800000        2
$1800000-$1900000        2
$1900000-$2000000        7
$2000000-$2100000        4
$2100000-$2200000        5
$2200000-$2300000        4
$2300000-$2400000        5
$2400000-$2500000        1
$2500000-$2600000        4
$2600000-$2700000        1
$2700000-$2800000        0
$2800000-$2900000        2
$2900000-$3000000        2
$3000000-$3100000        1
$3100000-$3200000        0
$3200000-$3300000        1
$3300000-$3400000        0
$3400000-$3500000        0
$3500000-$3600000        0
$36000

In [10]:
# Set pandas to display floats as regular notation
pd.set_option('display.float_format', '{:.2f}'.format)

# Define limits
upper_limit_price = 1500000  
upper_limit_dom = 730        

# Apply filters to remove rows with outliers
filtered_df = complete_df[
    (complete_df['median_list_price'] <= upper_limit_price) & 
    (complete_df['median_dom'] <= upper_limit_dom)
]

# Verify the changes
print(filtered_df[['median_list_price', 'median_dom']].describe())

       median_list_price  median_dom
count           38944.00    38944.00
mean           301247.78       71.08
std            146271.35       52.15
min             53000.00        0.00
25%            200000.00       38.00
50%            260400.00       59.00
75%            362325.00       90.00
max           1491600.00      727.00


In [11]:
# Engineer interaction features
complete_df['price_per_sqft_ratio'] = complete_df['median_sale_price'] / complete_df['median_ppsf']
complete_df['inventory_to_sales_ratio'] = complete_df['inventory'] / complete_df['homes_sold']

# Handle cases where division might result in NaN or infinite values
complete_df['price_per_sqft_ratio'] = complete_df['price_per_sqft_ratio'].fillna(0).replace(np.inf, 0)
complete_df['inventory_to_sales_ratio'] = complete_df['inventory_to_sales_ratio'].fillna(0).replace(np.inf, 0)

## Split the data into training (1/1/2014 - 2/28/2013) and testing sets (3/1/23 - 9/30/2024) and define our target. `median_sales_price`.

In [12]:
# Split the dataset into training and testing sets based on the Date column
train_data = complete_df[(complete_df['date'] >= '2013-01-01') & (complete_df['date'] <= '2023-2-28')]
test_data = complete_df[(complete_df['date'] >= '2023-03-01') & (complete_df['date'] <= '2024-09-30')]

# Check the split
print(f"Training data: {len(train_data)} rows")
print(f"Testing data: {len(test_data)} rows")

Training data: 30265 rows
Testing data: 4760 rows


In [13]:
# Drop the Date column along with redundant columns
columns_to_drop = ['date', 'case_shiller_index_mom', 'period_duration', 'table_id']  

# Define the train and test data
train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)

In [14]:
# Define the target variable (y)
y_train = train_data['median_sale_price']
y_test = test_data['median_sale_price']

# Define the features (X) by dropping only the target column 
X_train = train_data.drop(columns=['median_sale_price'])
X_test = test_data.drop(columns=['median_sale_price'])

print(X_train.columns == X_test.columns)  

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]


## Initialize, Train and fit our Random Forest model - Use Regressor as target variable `median_sales_price` is continuous.

In [15]:
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

In [16]:
# Fit the model on the training data
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [17]:
# Use the model to predict on the test data
y_pred = rf_model.predict(X_test)

## Evaluate Performance

In [18]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")

Mean Squared Error: 2174606187.4058824
Mean Absolute Error: 9055.399579831932
R² Score: 0.9443127098901758


In [19]:
train_score = rf_model.score(X_train, y_train)
test_score = rf_model.score(X_test, y_test)
print(f"Training Score: {train_score}")
print(f"Testing Score: {test_score}")

Training Score: 0.9975725207223667
Testing Score: 0.9443127098901758


In [20]:
# Calculate the top 25 features and sort in descending order
importances = rf_model.feature_importances_ 
feature_names = X_train.columns
sorted_indices = importances.argsort()[::-1]  

top_25_features = feature_names[sorted_indices][:25]  
top_25_features

Index(['median_list_price', 'price_per_sqft_ratio', 'median_list_ppsf',
       'median_ppsf', 'median_sale_price_yoy', 'index_sa',
       'median_sale_price_mom', 'property_type_Condo/Co-op', 'pending_sales',
       'inventory', 'inventory_yoy', 'median_ppsf_yoy', 'new_listings',
       'inventory_mom', 'homes_sold', 'median_ppsf_mom', 'avg_sale_to_list',
       'median_list_ppsf_yoy', 'avg_sale_to_list_yoy', 'median_dom',
       'pending_sales_yoy', 'homes_sold_mom', 'sold_above_list',
       'off_market_in_two_weeks_mom', 'sold_above_list_mom'],
      dtype='object')

In [21]:
# Set pandas display format to show six decimal places
pd.options.display.float_format = '{:.6f}'.format

# Create a DataFrame for the top 25 features and their importance scores
top_25_importance_df = pd.DataFrame({
    'Feature': top_25_features,  # Your top 25 features list
    'Importance': importances[sorted_indices][:25]  # Corresponding importance scores
})

# Sort by importance scores (descending order) for clarity
top_25_importance_df = top_25_importance_df.sort_values(by='Importance', ascending=False)

# Display the DataFrame
top_25_importance_df

,Feature,Importance
0,median_list_price,0.607232
1,price_per_sqft_ratio,0.143655
2,median_list_ppsf,0.113173
3,median_ppsf,0.101694
4,median_sale_price_yoy,0.008357
5,index_sa,0.004407
6,median_sale_price_mom,0.003169
7,property_type_Condo/Co-op,0.001703
8,pending_sales,0.001053
9,inventory,0.001044


## Redo Test to check model using `top_25_features` 

In [22]:
# Redefine X_train and X_test using the top_25_features
X_train_top25 = X_train[top_25_features]
X_test_top25 = X_test[top_25_features]

In [23]:
# Re-fit the model on the training data
rf_model.fit(X_train_top25, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [24]:
# Use the model to predict on the test data
y_pred_top25 = rf_model.predict(X_test_top25)

In [25]:
# Evaluate the model
mse_top25 = mean_squared_error(y_test, y_pred_top25)
mae_top25 = mean_absolute_error(y_test, y_pred_top25)
r2_top25 = r2_score(y_test, y_pred_top25)

# Print the results
print(f"Top 25 Mean Squared Error: {mse_top25}")
print(f"Top 25 Mean Absolute Error: {mae_top25}")
print(f"Top 25 R² Score: {r2_top25}")

Top 25 Mean Squared Error: 2114861817.265336
Top 25 Mean Absolute Error: 8519.361134453782
Top 25 R² Score: 0.9458426430301226


In [26]:
# Calculate scores using the top 25 features
train_score_top25 = rf_model.score(X_train_top25, y_train)
test_score_top25 = rf_model.score(X_test_top25, y_test)

# Print the results
print(f"Training Score (Top 25 Features): {train_score_top25}")
print(f"Testing Score (Top 25 Features): {test_score_top25}")

Training Score (Top 25 Features): 0.9978149086602324
Testing Score (Top 25 Features): 0.9458426430301226


## Apply findings from corrected model and run new model using Top 25 features.

In [31]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 300],  # Number of trees
    'max_depth': [10, 20, 30, None]      # Maximum depth of the trees
}

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           scoring='r2', cv=3, verbose=2, n_jobs=-1)

# Fit the grid search to the data (using top 25 features)
grid_search.fit(X_train_top25, y_train)

# Display the best parameters and the corresponding score
print("Best Parameters:", grid_search.best_params_)
print("Best R² Score:", grid_search.best_score_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best Parameters: {'max_depth': 20, 'n_estimators': 300}
Best R² Score: 0.9801565855588974
[CV] END .....................max_depth=10, n_estimators=100; total time=  24.1s
[CV] END ......................max_depth=20, n_estimators=50; total time=  21.1s
[CV] END .....................max_depth=20, n_estimators=100; total time=  45.1s
[CV] END ......................max_depth=30, n_estimators=50; total time=  22.2s
[CV] END .....................max_depth=30, n_estimators=100; total time=  44.8s
[CV] END ....................max_depth=None, n_estimators=50; total time=  21.9s
[CV] END ....................max_depth=None, n_estimators=50; total time=  22.1s
[CV] END ...................max_depth=None, n_estimators=100; total time=  43.0s
[CV] END .....................max_depth=10, n_estimators=200; total time=  49.3s
[CV] END .....................max_depth=20, n_estimators=200; total time= 1.5min
[CV] END .....................max_depth

## Retrain the model using the above setup with Top 25 features and applying the findings from the optimized parameters search

In [27]:
# Use the previous train and test setup on the top 25 features
X_train_top25 = X_train[top_25_features]
X_test_top25 = X_test[top_25_features]

In [28]:
best_rf_model = RandomForestRegressor(max_depth=20, n_estimators=300, random_state=42, n_jobs=-1)

In [29]:
# Re-train the model on the updated features set
best_rf_model.fit(X_train_top25, y_train)

RandomForestRegressor(max_depth=20, n_estimators=300, n_jobs=-1,
                      random_state=42)

In [30]:
# Updated predictions using the top 25 and optimized model
y_pred_optimized = best_rf_model.predict(X_test_top25)

In [31]:
# perform updated test results with the optimized model run on the top 25 features
mse_optimized = mean_squared_error(y_test, y_pred_optimized)
mae_optimized = mean_absolute_error(y_test, y_pred_optimized)
r2_optimized = r2_score(y_test, y_pred_optimized)

print(f"Optimized Mean Squared Error: {mse_optimized}")
print(f"Optimized Mean Absolute Error: {mae_optimized}")
print(f"Optimized R² Score: {r2_optimized}")

Optimized Mean Squared Error: 2030815552.3477826
Optimized Mean Absolute Error: 8234.135995214614
Optimized R² Score: 0.9479948988106021


In [37]:
# Take optimization step further and check for min samples split and min samples leaf

# Updated parameter grid for Number of trees, tree depth, min samples required before splitting and min samples per leaf mode
param_grid = {
    'n_estimators': [100, 200, 300],      
    'max_depth': [10, 20, None],          
    'min_samples_split': [2, 10, 20],     
    'min_samples_leaf': [1, 5, 10]       
}

# Initialize Random Forest Regressor
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Setup GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           scoring='r2', cv=3, verbose=2, n_jobs=-1)

# Perform the grid search
grid_search.fit(X_train_top25, y_train)

# Display the best parameters and corresponding score
print("Best Parameters:", grid_search.best_params_)
print("Best R² Score:", grid_search.best_score_)


Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best R² Score: 0.9801565855588974
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time= 1.3min
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=20, n_estimators=100; total time=  27.4s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=20, n_estimators=300; total time= 1.4min
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=10, n_estimators=200; total time=  55.1s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=20, n_estimators=200; total time=  55.8s
[CV] END max_depth=10, min_samples_leaf=10, min_samples_split=2, n_estimators=200; total time=  55.8s
[CV] END max_depth=10, min_samples_leaf=10, min_samples_split=10, n_estimators=300; total time= 1.4min
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total

## Apply the advanced optimized model to the `top_25`

In [32]:
# Use the previous train and test setup on the top 25 features
X_train_top25 = X_train[top_25_features]
X_test_top25 = X_test[top_25_features]

In [33]:
# Create the new model
final_rf_model = RandomForestRegressor(max_depth=20, min_samples_leaf=1, 
                                       min_samples_split=2, n_estimators=300, 
                                       random_state=42, n_jobs=-1)

In [34]:
# Re-train the model
final_rf_model.fit(X_train_top25, y_train)

RandomForestRegressor(max_depth=20, n_estimators=300, n_jobs=-1,
                      random_state=42)

In [35]:
# Make the new predictions 
y_pred_final = final_rf_model.predict(X_test_top25)

In [36]:
# Score the final model
mse_final = mean_squared_error(y_test, y_pred_final)
mae_final = mean_absolute_error(y_test, y_pred_final)
r2_final = r2_score(y_test, y_pred_final)

print(f"Final Mean Squared Error: {mse_final}")
print(f"Final Mean Absolute Error: {mae_final}")
print(f"Final R² Score: {r2_final}")

Final Mean Squared Error: 2030815552.3477826
Final Mean Absolute Error: 8234.135995214612
Final R² Score: 0.9479948988106021


In [37]:
# Calculate the difference and round to 2 decimal places to mirror actual dollar figure format

# Set up the rounding to 2 decimal places
actual_values = np.round(y_test.values, 2)  
predicted_values = np.round(y_pred_final, 2)  
differences = np.round(actual_values - predicted_values, 2)  

# Create the results DataFrame
results = pd.DataFrame({
    'Actual': actual_values,
    'Predicted': predicted_values,
    'Difference': differences
})

# Save the results to a CSV file
results.to_csv('Resources/rf_optimized_1_results.csv', index=False)

# Confirm the saved file contains properly rounded values
print("Results saved with Actual, Predicted and Difference columns!")

Results saved with Actual, Predicted and Difference columns!


In [39]:
from joblib import dump

# Save the trained model to a file
dump(final_rf_model, 'Resources/final_rf_model.joblib')

print("Model saved successfully!")

Model saved successfully!


In [ ]:
from joblib import load

# Load the model from the file
loaded_model = load('Resources/final_rf_model.joblib')

print("Model loaded successfully!")
